In [ ]:
from datetime import datetime
from dateutil import relativedelta

now = datetime(year=2023, month=12, day=18, hour=12, minute=18, second=0)

print('현재시간:' + str(now))
print('------------월 연산-------------')
print(now + relativedelta.relativedelta(month=1))
print(now.replace(month=1))

In [ ]:
import snowflake.connector
import pandas as pd

# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='hdhsdwload',                # 스노우플레이크 사용자명
    password='Ghatyvld1!',            # 비밀번호
    account='dz48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_LOAD_WH'          # 스키마 이름
)

try:\
    
    
    # 커서 생성
    cursor = conn.cursor()

    # SQL 쿼리 실행
    query = "SELECT COUNT(*) FROM DW_LOAD_DB.HDHS_CU.CU_ITNT_CUST_GRD_1_INF"  # 조회할 테이블과 조건
    query2 = "alter stage DW_LOAD_DB.CONFIG.STG_AWS_S3 refresh"  # 조회할 테이블과 조건
    query3 = f"""
        SELECT 
           *
        FROM 
            DW_LOAD_DB.CONFIG.V_S3_LIST A 
        """
    cursor.execute(query)
    print (cursor.fetchall())
    
    cursor.execute(query2)
    print (type(cursor.fetchone()[1]))
    
    colums = [row[0] for row in cursor.fetchall()]
    df = pd.read_sql(query3, conn)

    # 결과 출력

finally:
    # 연결 닫기
    cursor.close()
    conn.close()
    
df

In [ ]:
df =pd.DataFrame(df)

In [ ]:
df

In [ ]:
df.to_parquet('df.parquet',engine='pyarrow', index=False)

In [ ]:
df['TABLE_NAME'][0]

In [ ]:
# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='hdhsdwload',                # 스노우플레이크 사용자명
    password='Ghatyvld1!',            # 비밀번호
    account='dz48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_LOAD_WH'          # 스키마 이름
)

cursor = conn.cursor()

# for i,row in df.iterrows():
#     if df['TABLE_SCHEMA'][i] == "HDHS_OD" and df['TABLE_NAME'][i] != "OD_BASKT_INF":
#         
#         query5 = f"""
#         CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB('DW_LOAD_DB','HDHS_OD','{df['TABLE_NAME'][i]}','{df['FILE_NAME']}','20241223120000','20241223150000')
#         """
#         print(query5)
#         cursor.execute(query5)
#         print(cursor.fetchall())
        
query5 = f""" 
    CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
        'DW_LOAD_DB',
        'HDHS_OD',
        '{df['TABLE_NAME'][1]}',""" + str("''2024/12/23/CU_ARS_LDIN_MST_CRYPT_20241223_030000.csv','2024/12/23/20241223_040000.csv''") +""",
        '20241223120000',
        '20241223150000'
    )
    """
print(query5)
cursor.execute(query5)
print(cursor.fetchall())
        
        
cursor.close()
conn.close() 

In [ ]:
fmrom airflow.providers.snowflake.operators.snowflake import SnowflakeOperator

CALL_PROCEDURE_QUERY = """
CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
    'DW_LOAD_DB',
    'HDHS_OD',
    '{{ params.table_name }}',
    '{{ params.file_name }}',
    '{{ params.start_time }}',
    '{{ params.end_time }}'
)
"""

call_procedure = SnowflakeOperator(
    task_id='call_snowflake_procedure',
    snowflake_conn_id='my_snowflake_conn',
    sql=CALL_PROCEDURE_QUERY,
    params={
        'table_name': {df['TABLE_NAME'][1]},
        'file_name': {df['FILE_NAME'][1]},
        'start_time': '20241223120000',
        'end_time': '20241223150000'
    },
)

print(call_procedure)


In [6]:
import pandas as pd

src_columns =['PROC_TIME','TRED','CALL_ID','MESG','Q_WAIT_TIME','STON','VDN_DIAL_NO','UCID','IN_CALL_NO']
tgt_columns =['UCID','ACD','PROC_DATE','PROC_TIME','TRED','CALL_ID','MESG','Q_WAIT_TIME','STON','VDN_DIAL_DVCD','VDN_DIAL_NO','IN_CALL_NO','ETL_DTM']
file_path="hmall_1225.dmp"
# 파일을 Pandas DataFrame으로 변환
try:
    # 쉼표로 구분된 데이터를 읽기 (UTF-8 또는 EUC-KR 등의 인코딩을 시도)
    df = pd.read_csv(file_path, encoding='utf-8', header=None)
except UnicodeDecodeError:
    # 만약 UTF-8로 읽히지 않으면 EUC-KR로 재시도
    df = pd.read_csv(file_path, encoding='euc-kr', header=None)

df = df.iloc[:, :-3]

df.columns = src_columns

df



,PROC_TIME,TRED,CALL_ID,MESG,Q_WAIT_TIME,STON,VDN_DIAL_NO,UCID,IN_CALL_NO
0,00:00:00.146,2140,c6100,상담연결(AgentVDN->Agent),BSR,i01093101121,v60324,U10001044731703429962,A01093101121
1,00:00:01.146,2328,c6097,상담연결(AgentVDN->Agent),,i01084865650,v60717,U10002060531703429968,A01084865650
2,00:00:02.114,22a8,c6090,상담연결(AgentVDN->Agent),,i01029385307,v60105,U10002060901703429993,A01029385307
3,00:00:02.614,17e4,c6010,호종료,,s3167,v,U,A
4,00:00:02.614,205c,c5801,수신(Inbound),,s3167,v3416,U10002058011703429734,A01073785912
...,...,...,...,...,...,...,...,...,...
118551,23:45:58.331,0dc8,c2656,상담연결(AgentVDN->Agent),,i10512,v62044,U10002026541735137940,A0618633452
118552,23:58:01.657,2fc0,c,IVR상담연결(IVR->CTI),,i10085,v64601,U,A01021140996
118553,23:58:44.695,30ec,c,IVR상담연결(IVR->CTI),,i10184,v62036,U,A01075774322
118554,23:58:46.398,3370,c2900,상담연결(AgentVDN->Agent),,i10184,v62036,U10002028831735138697,A01075774322


In [7]:
from datetime import timedelta

def create_target_dataframe(df, tgt_columns, execution_date):
    # 실행 날짜 계산
    proc_date = (execution_date - timedelta(days=1)).strftime('%Y%m%d')
    proc_date_yyyymmmm = proc_date[:8]  # YYYYMM 포맷으로 가져오기
    
    # 데이터 변환
    transformed_data = {
        'UCID': df['UCID'].str[1:21],  # SUBSTR(UCID,2,20)
        'ACD': 2,                      # 고정값
        'PROC_DATE': proc_date_yyyymmmm, # execution_date - 1일에서 YYYYMMMM 추출
        'PROC_TIME': df['PROC_TIME'].str.lstrip(),  # LTRIM(PROC_TIME)
        'TRED': df['TRED'],            # 그대로 유지
        'CALL_ID': df['CALL_ID'].str[1:6],  # SUBSTR(CALL_ID,2,5)
        'MESG': df['MESG'],            # 그대로 유지
        'Q_WAIT_TIME': df['Q_WAIT_TIME'].apply(
            lambda x: x[1:] if x.startswith('Q') else x  # IIF 조건 적용
        ),
        'STON': df['STON'].str[1:5],   # SUBSTR(STON,2,4)
        'VDN_DIAL_DVCD': df['VDN_DIAL_NO'].str[0].str.upper(),  # UPPER(SUBSTR(VDN_DIAL_NO,1,1))
        'VDN_DIAL_NO': df['VDN_DIAL_NO'].str[1:],  # SUBSTR(VDN_DIAL_NO,2,LENGTH(VDN_DIAL_NO))
        'IN_CALL_NO': df['IN_CALL_NO'].str[1:],  # SUBSTR(IN_CALL_NO,2,LENGTH(IN_CALL_NO))
        'ETL_DTM': execution_date.strftime('%Y-%m-%d %H:%M:%S')  # execution_date
    }
    target_df = pd.DataFrame(transformed_data, columns=tgt_columns)
    return target_df

execution_date = pd.Timestamp('2023-12-26')

# 변환된 데이터프레임 생성
result_df = create_target_dataframe(df, tgt_columns, execution_date)

# 결과 확인
result_df

,UCID,ACD,PROC_DATE,PROC_TIME,TRED,CALL_ID,MESG,Q_WAIT_TIME,STON,VDN_DIAL_DVCD,VDN_DIAL_NO,IN_CALL_NO,ETL_DTM
0,10001044731703429962,2,20231225,00:00:00.146,2140,6100,상담연결(AgentVDN->Agent),BSR,0109,V,60324,01093101121,2023-12-26 00:00:00
1,10002060531703429968,2,20231225,00:00:01.146,2328,6097,상담연결(AgentVDN->Agent),,0108,V,60717,01084865650,2023-12-26 00:00:00
2,10002060901703429993,2,20231225,00:00:02.114,22a8,6090,상담연결(AgentVDN->Agent),,0102,V,60105,01029385307,2023-12-26 00:00:00
3,,2,20231225,00:00:02.614,17e4,6010,호종료,,3167,V,,,2023-12-26 00:00:00
4,10002058011703429734,2,20231225,00:00:02.614,205c,5801,수신(Inbound),,3167,V,3416,01073785912,2023-12-26 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118551,10002026541735137940,2,20231225,23:45:58.331,0dc8,2656,상담연결(AgentVDN->Agent),,1051,V,62044,0618633452,2023-12-26 00:00:00
118552,,2,20231225,23:58:01.657,2fc0,,IVR상담연결(IVR->CTI),,1008,V,64601,01021140996,2023-12-26 00:00:00
118553,,2,20231225,23:58:44.695,30ec,,IVR상담연결(IVR->CTI),,1018,V,62036,01075774322,2023-12-26 00:00:00
118554,10002028831735138697,2,20231225,23:58:46.398,3370,2900,상담연결(AgentVDN->Agent),,1018,V,62036,01075774322,2023-12-26 00:00:00


In [5]:
dtm = '20250122000000'

In [10]:
import datetime
dtm_obj = datetime.datetime.strptime(dtm, '%Y%m%d%H%M%S')
formatted_dtm = dtm_obj.strftime('%Y-%m-%d %H:%M:%S')

In [12]:
print(formatted_dtm)
print(dtm_obj)
print(formatted_dtm)
print(dtm_obj)

2025-01-22 00:00:00
2025-01-22 00:00:00


In [14]:
# Given text
text = """DW_BM	BCU_CUST_MST
DW_BM	BCU_CUST_REFI_CTEL_INF
DW_BM	BCU_EMAIL_ADR_REFI_CUST_MST
DW_BM	BCU_SMS_RCV_AGR_CUST_INF
DW_BM	BMK_CUST_MKTG_UTLZ_AGR_INF
DW_BM	BOD_ORD_DC_DTL
DW_HPNT	PC_PTCO_ITEM_MST
DW_HPNT	PC_PTCO_MEM_INF
DW_HPNT	PC_PTCO_SALE_DTL
DW_HPNT	PC_PTCO_SALE_STLM_DTL
DW_HSIS	HES_RNTL_ARLT_DTL
DW_PB	POD_ACPT_META_ORD_ANAL_DLU_FCT_03
DW_RM	RAR_BITM_SELL_ETC_REAL_SMR
DW_RM	RCA_MDA_AREA_CALL_HOU_FCT
DW_BM	BCU_CUST_STAT_MST
DW_BM	BCU_PUSH_RCV_AGR_CUST_INF
DW_ETC	DW_USE_RATIO
DW_HSIS	HES_BRND_CTPF_RATE_ETC_DTL
DW_PB	PAR_PHDS_ARLT_DLU_FCT
DW_PB	PAR_RNTL_ARLT_DLU_FCT_02
DW_PB	PMA_EVNT_ANAL_DLU_FCT
DW_PB	POD_ORD_ANAL_DLU_FCT_03
DW_RM	RAR_CTPF_RATE_ETC_DTL
DW_RM	RCA_THDY_TMR_FCT
DW_RM	RCU_CUST_ONLN_ACSS_INF
DW_RM	RIA_DTBRC_ORD_REAL_FCT_02
DW_RM	RPS_DPRCH_SIS_DTL
DW_RM	RCU_BITM_CNSL_SMR
DW_RM	RIA_PREV_QLTY_EVAL_DLU_SMR
DW_RM	ROD_DPRCH_ORD_DTL
DW_RM	ROD_GRNT_SVMT_DLU_SMR
DW_PB	PAR_PHDS_ARLT_DLU_FCT_02
DW_RM	RAR_HMALL_BMNG_ARLT_DLU_SMR
DW_RM	RAR_PGM_UITM_FCT
DW_PB	POD_ACPT_META_ORD_ANAL_DLU_FCT_02
DW_RM	RAR_BMNG_ARLT_DLU_SMR
DW_RM	RAR_PGM_BUY_CUST_FCT
DW_RM	RCU_MKTG_AGR_SMS_DTL_TMP
DW_RM	RIA_TV_BITM_ORD_FCT
DW_RM	ROD_AREA_ORD_SMR
DW_PB	POD_ACPT_DLU_ORD_SMR
DW_RM	ROD_ORD_SVMT_STLM_DTL
DW_HSIS	HES_EXP_SWRT_DTL
DW_RM	RCA_DRCS_ARLT_DLU_FCT
DW_RM	RIA_ITEM_ANAL_FCT
DW_RM	RCU_CUST_MOTH_ORD_SMR
DW_BM	BCM_POST_NO_SI_DO_INF
DW_BM	BCU_CUST_REFI_CTEL_ERR_INF
DW_BM	BCU_CUST_TNDC_INF
DW_BM	BCU_HMALL_CUST_MST
DW_BM	BIM_ITEM_MST
DW_BM	BOD_ORD_DTL
DW_HPNT	PC_PTCO_BASKT_INF
DW_HPNT	PC_PTCO_ITEM_SALE_DTL
DW_HSIS	HES_EXP_SWRT_ETC_DTL
DW_PB	PCU_CUST_AGR_MOTH_FCT_01
DW_PB	PCU_CUST_AGR_MOTH_FCT_02
DW_RM	RAR_BMNG_ARLT_DLU_FCT
DW_RM	RAR_HS_BMNG_ARLT_DLU_SMR
DW_RM	RDM_HMALL_SECT_DPTH_DIM
DW_ETC	MSOF_AREA_SO_DMC_CLOG
DW_ETC	TSMS_AGENT_MESSAGE_DORMANT
DW_HSIS	HES_EXP_SWRT_MLB_DTL
DW_PB	PAR_SHPL_BROD_ARLT_DLU_FCT
DW_PB	PCU_CUST_ANAL_FCT_01
DW_PB	PMA_COPN_ANAL_DLU_FCT_01
DW_RM	RAR_EXP_SWRT_ONLN_DTL
DW_RM	RAR_ORD_PNTM_SELL_PTC_ETC_REAL_SMR
DW_RM	RAR_REAL_SWRT_DTL
DW_RM	RCA_TMR_CALL_DLU_FCT
DW_RM	RCU_TGC_MOTH_SMR
DW_RM	RDM_ALLI_REF_CH_DIM
DW_RM	RIA_NEW_ITEM_WKU_SMR
DW_RM	RIA_NEW_RCMM_ITEM_INF
DW_RM	RMA_BROD_COPN_USE_DTL
DW_RM	RMA_WINT_INSM_CMISR_ETC_INF
DW_RM	RMA_WINT_INSM_CMISR_INF
DW_RM	RPD_DLV_TRC_DTL
DW_RM	RPD_DPRCH_ITEM_MOTH_STCK_INF
DW_RM	RAR_BRND_CTPF_RATE_DTL
DW_RM	RAR_EXP_SWRT_DTL
DW_RM	RAR_SELL_PTC_ETC_REAL_SMR
DW_RM	RIA_BITM_ORD_EXP_FCT
DW_RM	RMA_BROD_COPN_PBLC_DTL
DW_RM	RPS_DPRCH_SCO_DTL
DW_RM	RAR_BMNG_ARLT_DLU_ACPT_META_FCT_02
DW_GCON	CMS_REPORT
DW_RM	RDM_MD_ORGN_DIM
DW_RM	RIA_BITM_ORD_CUST_FCT
DW_RM	ROD_CUST_PET_GRD_INF
DW_PB	PPD_OSHP_TKTM_DYS_FCT
DW_RM	RAR_BMNG_ARLT_DLU_ACPT_META_FCT
DW_RM	RCU_HMALL_CUST_GRD_TMP
DW_RM	RDM_BITM_GRP_DIM
DW_RM	RIA_RVCO_NEW_ITEM_INF
DW_RM	ROD_OORD_DLU_SMR
DW_RM	ROD_SO_SALE_DLU_SMR
DW_RM	ROD_USE_SVMT_DLU_SMR
DW_RM	RIA_BITM_ORD_EXP_FCT_02
DW_RM	RIA_KWRD_PGM_ACHV_RATE_SMR
DW_RM	RPS_DPRCH_IO_DTL
DW_BM	BMK_CUST_MKTG_AGR_EMAIL_DTL
DW_BM	BOD_ORD_CTPF_VACO_DTL
DW_BM	BOD_ORD_PTC
DW_BM	BOD_RNTL_ORD_DTL
DW_ETC	DAILY_PRCDR_BAK
DW_ETC	DAU_SMS_DROP
DW_HPNT	PC_PTCO_WRK_INF
DW_PB	PMA_COPN_ANAL_DLU_FCT_02
DW_RM	RAR_BRND_CTPF_RATE_ETC_DTL
DW_RM	RAR_DPTS_ITEM_REG_SMR
DW_RM	RCU_CUST_BUY_CHRTR_MOTH_INF
DW_RM	RCU_EVNT_PTCP_DLU_SMR
DW_RM	RIA_BITM_ORD_REAL_FCT_02
DW_RM	RIA_DTBRC_ORD_REAL_FCT
DW_HSIS	HES_EXP_SWRT_ONLN_ETC_DTL
DW_PB	DLV_WTHDW_CNT_SMR
DW_PB	PAR_RNTL_ARLT_DLU_FCT
DW_PB	PIA_BROD_ANAL_DLU_FCT_01
DW_PB	POD_ORD_ANAL_DLU_FCT_01
DW_RM	RAR_BITM_MDA_SELL_REAL_SMR
DW_RM	RAR_CTPF_RATE_HMALL_DTL
DW_RM	RAR_EXP_SWRT_HMALL_DTL
DW_RM	RAR_REAL_SWRT_ONLN_ETC_DTL
DW_RM	RAR_RNTL_ARLT_DTL
DW_RM	RAR_SCWD_DLU_ORD_ARLT_SMR
DW_RM	RCU_CUST_MDA_ORD_INF
DW_RM	RCU_HMALL_CUST_GRD_DTL
DW_RM	RDM_SELL_MDA_DIM
DW_RM	RIA_BFMT_PRJ_DTL
DW_RM	RIA_DTBRC_ORD_EXP_FCT_02
DW_RM	ROD_CUST_PET_GRD_HIS
DW_RM	RPD_DLV_FORM_DLU_SMR
DW_RM	RCU_BITM_DT_ARLT_SMR
DW_RM	RCU_CUST_BUY_CHRTR_INF
DW_RM	RCU_HMALL_CUST_GRD_MOTH_DTL
DW_RM	RDM_SELL_MDA_CSF_DIM
DW_RM	RMA_HMALL_COPN_PBLC_DTL
DW_PB	PAR_HMALL_DLU_ARLT_SMR
DW_PB	POD_ACPT_META_ORD_ANAL_DLU_FCT_01
DW_RM	RAR_BITM_ORD_PNTM_SELL_PTC_REAL_SMR
DW_RM	RAR_VEN_CNTB_RATE_SMR
DW_RM	RCU_GGC_MOTH_SMR
DW_RM	RAR_REAL_SWRT_ETC_DTL
DW_RM	RIA_BITM_ORD_REAL_FCT
DW_RM	RIA_BITM_SELL_MDA_FCT
DW_RM	ROD_SELL_MDA_SVMT_STLM_DTL
DW_RM	ROD_SO_OPER_MOTH_SMR
DW_RM	ROD_VEN_CUST_ORD_DTL
DW_RM	RPS_DRCT_PRCH_LOSS_DTL
BI_DEV	ACCLN_SALE_DTL
BI_DEV	DLV_OSHP_WTDW_DTL
DW_BM	BAR_ITEM_HNDL_ARLT_DLINE_DTL
DW_BM	BBD_BFMT_HOPE_DTL
DW_BM	BCU_CAMP_TRGT_CUST_MST
DW_BM	BMK_CUST_MKTG_AGR_HIS
DW_BM	BMK_CUST_VLID_TERM_EMAIL_DTL
DW_BM	BOD_ORD_STLM_DTL
DW_ETC	DAU_SMS_DORMANT
DW_HPNT	PC_PTCO_ITEM_CTGR_INF
DW_PB	PAR_INSU_ARLT_DLU_FCT
DW_PB	PCU_CUST_ANAL_FCT_02
DW_RM	RAR_CTPF_RATE_DTL
DW_RM	RAR_SRCN_MD_ARLT_DLU_SMR
DW_RM	RCA_MDA_AREA_CALL_DLU_FCT
DW_RM	RIA_KWRD_ARLT_RANK_SMR
DW_BM	BMK_CUST_MKTG_AGR_MST
DW_BM	BMK_CUST_MKTG_STAT_HIS
DW_ETC	TSMS_AGENT_MESSAGE_DROP
DW_HSIS	HES_DRCT_PRCH_LOSS_DTL
DW_HSIS	HES_EXP_SWRT_ONLN_DTL
DW_RM	RAR_AFCR_ORD_ARLT_SMR
DW_RM	RAR_BITM_SELL_PTC_REAL_SMR
DW_RM	RAR_BITM_SELL_REAL_SMR
DW_RM	RAR_EXP_SWRT_ETC_DTL
DW_RM	RCU_CRM_MOTH_KPI_FCT
DW_RM	RCU_CUST_ONLN_ACSS_DLU_DTL
DW_RM	RCU_HS_CUST_MOTH_SMR
DW_RM	RCU_MOTH_MDA_CUST_D3_SMR
DW_RM	RCU_TCS_CUST_HIS_TMP
DW_RM	RIA_DTBRC_ORD_EXP_FCT
DW_RM	RPD_DLV_FORM_OSHP_DLU_FCT
DW_RM	RPD_DPRCH_ITEM_STCK_INF
DW_RM	RAR_CRDC_SEVT_RST_DLU_SMR
DW_RM	RAR_EXP_SWRT_MLB_DTL
DW_RM	RAR_EXP_SWRT_ONLN_ETC_DTL
DW_RM	RAR_REAL_SWRT_ONLN_DTL
DW_RM	RCU_GGC_TMP
DW_RM	RCU_MOTH_MDA_CUST_SMR
DW_BM	BCM_HMALL_SECT_DPTH_MST
DW_RM	RAR_BFMT_DLU_FCT
DW_PB	PAR_SO_ARLT_DLU_FCT
DW_PB	PIA_BROD_ANAL_DLU_FCT_02
DW_PB	POD_ACPT_ORD_ANAL_DLU_FCT
DW_RM	RCU_CRM_DLU_KPI_FCT
DW_RM	ROD_DPRCH_BROD_ORD_DTL
DW_RM	RPD_DPRCH_EUPDN_INF
DW_RM	RAR_MDA_ORD_PNTM_SELL_ETC_REAL_SMR
DW_RM	RAR_MDA_SELL_ETC_REAL_SMR
DW_RM	RAR_STLM_STAT_DLU_SMR
DW_RM	RCU_CUST_PRFR_ITEM_L_CSF_INF
DW_RM	RAR_BITM_MDA_ORD_PNTM_SELL_REAL_SMR
DW_RM	RDM_BSIC_YR_AGE_DIM
DW_RM	RDM_SO_CH_MOTH_DIM
DW_RM	RIA_BITM_ORD_FCT
DW_RM	RIA_ITEM_ANAL_FCT_02
DW_RM	RMA_HMALL_COPN_USE_DTL
DW_RM	ROD_CUST_PET_GRD_INF_TMP
DW_RM	ROD_PGM_SVMT_STLM_DTL
DW_RM	RPS_SO_SALE_DLU_DLINE_DTL"""

# Split and format the text
formatted_text = ",\n".join([f"('{line.split()[0]}', '{line.split()[1]}', 'N')" for line in text.split("\n")])

# Display result
print(formatted_text)


('DW_BM', 'BCU_CUST_MST', 'N'),
('DW_BM', 'BCU_CUST_REFI_CTEL_INF', 'N'),
('DW_BM', 'BCU_EMAIL_ADR_REFI_CUST_MST', 'N'),
('DW_BM', 'BCU_SMS_RCV_AGR_CUST_INF', 'N'),
('DW_BM', 'BMK_CUST_MKTG_UTLZ_AGR_INF', 'N'),
('DW_BM', 'BOD_ORD_DC_DTL', 'N'),
('DW_HPNT', 'PC_PTCO_ITEM_MST', 'N'),
('DW_HPNT', 'PC_PTCO_MEM_INF', 'N'),
('DW_HPNT', 'PC_PTCO_SALE_DTL', 'N'),
('DW_HPNT', 'PC_PTCO_SALE_STLM_DTL', 'N'),
('DW_HSIS', 'HES_RNTL_ARLT_DTL', 'N'),
('DW_PB', 'POD_ACPT_META_ORD_ANAL_DLU_FCT_03', 'N'),
('DW_RM', 'RAR_BITM_SELL_ETC_REAL_SMR', 'N'),
('DW_RM', 'RCA_MDA_AREA_CALL_HOU_FCT', 'N'),
('DW_BM', 'BCU_CUST_STAT_MST', 'N'),
('DW_BM', 'BCU_PUSH_RCV_AGR_CUST_INF', 'N'),
('DW_ETC', 'DW_USE_RATIO', 'N'),
('DW_HSIS', 'HES_BRND_CTPF_RATE_ETC_DTL', 'N'),
('DW_PB', 'PAR_PHDS_ARLT_DLU_FCT', 'N'),
('DW_PB', 'PAR_RNTL_ARLT_DLU_FCT_02', 'N'),
('DW_PB', 'PMA_EVNT_ANAL_DLU_FCT', 'N'),
('DW_PB', 'POD_ORD_ANAL_DLU_FCT_03', 'N'),
('DW_RM', 'RAR_CTPF_RATE_ETC_DTL', 'N'),
('DW_RM', 'RCA_THDY_TMR_FCT', 'N'),
('DW_RM

In [15]:
x = [200]
x = x[1:0]
x

[]

In [117]:
import snowflake.connector
import pandas as pd

# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='MWAA_API',                # 스노우플레이크 사용자명
    password='ghaTyvld1!',            # 비밀번호
    account='KE08776.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_ETL_WH'          # 스키마 이름
)

snow_cursor = conn.cursor()

[2025-02-12T08:15:01.293+0900] {connection.py:413} INFO - Snowflake Connector for Python Version: 3.12.2, Python Version: 3.11.10, Platform: Windows-10-10.0.22631-SP0
[2025-02-12T08:15:01.295+0900] {connection.py:1196} INFO - Connecting to GLOBAL Snowflake domain
[2025-02-12T08:15:01.296+0900] {connection.py:1277} INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


In [116]:
from datetime import datetime

table_list_query = "select PGMID,STARTTIME,ENDTIME from DW_ETL_DB.CONFIG.JOB_RESULT LIMIT 5"
snow_cursor.execute(table_list_query)
x = snow_cursor.fetchall()[0]
x

[2025-02-11T18:15:49.027+0900] {cursor.py:1156} INFO - Number of results in first chunk: 5


('SP_RIA_BITM_SELL_MDA_FCT',
 datetime.datetime(2025, 2, 10, 9, 38, 10, 209260),
 datetime.datetime(2025, 2, 10, 9, 38, 21, 283454))

In [123]:
xd = 'Null'
table_list_query = f"""INSERT INTO DW_ETL_DB.DW_ETC.ORA_SNOW_DATA_PIPE_VAL (ORA_START_TIME)
                                VALUES ({xd})
                                """
snow_cursor.execute(table_list_query)

In [66]:
 for procedure in query_list:
    snow_query = f"SELECT * FROM DW_ETL_DB.CONFIG.JOB_RESULT WHERE PGMID = '{procedure[0]}' AND JBPMT = '{procedure[1]}' AND STARTTIME >= TO_DATE('2025-02-10 00:00:00', 'YYYY-MM-DD HH24:MI:SS')"
    snow_cursor.execute(snow_query)
    qlist = [(row[0],row[1]) for row in snow_cursor.fetchall()]
print(qlist)
    
    

[2025-02-10T13:08:18.404+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.443+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.508+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.571+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.620+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.667+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.705+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.745+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.792+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.841+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-10T13:08:18.879+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1

In [105]:
schema = "ODS_TMS"
table_name = "TMS_APP_USER_LIST"
columns = ["APP_GRP_ID", "CUST_ID", "LAST_DEVICE_ID", "REG_DATE", "SITE_ID", "UPT_DATE"]
pk_columns = ["CUST_ID", "UPT_DATE"]

In [106]:


merge_query = f"""
    MERGE INTO {schema}.{table_name} AS target
    USING (SELECT {', '.join(['%s' for _ in columns])}) AS source ({', '.join(columns)})
    ON {' AND '.join([f"target.{col} = source.{col}" for col in pk_columns])}
    WHEN MATCHED THEN UPDATE SET {', '.join([f"target.{col} = source.{col}" for col in columns if col not in pk_columns])}
    WHEN NOT MATCHED THEN INSERT ({', '.join(columns)})
    VALUES ({', '.join(['source.' + col for col in columns])})
"""

In [108]:
print(merge_query)


    MERGE INTO ODS_TMS.TMS_APP_USER_LIST AS target
    USING (SELECT %s, %s, %s, %s, %s, %s) AS source (APP_GRP_ID, CUST_ID, LAST_DEVICE_ID, REG_DATE, SITE_ID, UPT_DATE)
    ON target.CUST_ID = source.CUST_ID AND target.UPT_DATE = source.UPT_DATE
    WHEN MATCHED THEN UPDATE SET target.APP_GRP_ID = source.APP_GRP_ID, target.LAST_DEVICE_ID = source.LAST_DEVICE_ID, target.REG_DATE = source.REG_DATE, target.SITE_ID = source.SITE_ID
    WHEN NOT MATCHED THEN INSERT (APP_GRP_ID, CUST_ID, LAST_DEVICE_ID, REG_DATE, SITE_ID, UPT_DATE)
    VALUES (source.APP_GRP_ID, source.CUST_ID, source.LAST_DEVICE_ID, source.REG_DATE, source.SITE_ID, source.UPT_DATE)



In [159]:
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta

# 생성할 더미 데이터 개수
num_rows = 100

# 랜덤 데이터 생성 함수
def random_date(start, end):
    return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

start_date = datetime(2023, 1, 1)
end_date = datetime(2025, 2, 12)

data = {
    "DEVICE_ID": [i for i in range(1, num_rows + 1)],
    "SITE_ID": [random.randint(1, 10) for _ in range(num_rows)],
    "APP_GRP_ID": [random.randint(1, 5) for _ in range(num_rows)],
    "APP_ID": [random.randint(1, 100) for _ in range(num_rows)],
    "CUST_ID": [str(random.randint(100000, 999999)) if random.random() > 0.2 else None for _ in range(num_rows)],
    "LOGIN_YN": [random.choice(['Y', 'N']) for _ in range(num_rows)],
    "UUID": [str(uuid.uuid4()) for _ in range(num_rows)],
    "SESS_FLAG": [random.choice(['YYN', 'YNN', 'NNN']) for _ in range(num_rows)],
    "TOKEN": [str(uuid.uuid4()).replace('-', '') if random.random() > 0.3 else None for _ in range(num_rows)],
    "NOTI_FLAG": [random.choice(['Y', 'N']) for _ in range(num_rows)],
    "BMKT_FLAG": [random.choice(['Y', 'N']) for _ in range(num_rows)],
    "APP_VER": [f"{random.randint(1, 10)}.{random.randint(0, 9)}.{random.randint(0, 9)}" for _ in range(num_rows)],
    "OS": [random.choice(['A', 'I']) for _ in range(num_rows)],
    "OS_VER": [f"{random.randint(5, 15)}.{random.randint(0, 9)}" for _ in range(num_rows)],
    "DEVICE": [random.choice(['Samsung Galaxy S21', 'iPhone 13', 'Google Pixel 6', 'OnePlus 9']) for _ in range(num_rows)],
    "DEL_YN": [random.choice(['Y', 'N']) for _ in range(num_rows)],
    "REG_DATE": [random_date(start_date, end_date) for _ in range(num_rows)],
    "UPT_DATE": [random_date(start_date, end_date) if random.random() > 0.5 else None for _ in range(num_rows)],
    "NIGHT_FLAG": [random.choice(['Y', 'N']) if random.random() > 0.5 else None for _ in range(num_rows)]
}

df = pd.DataFrame(data)

for col in df.select_dtypes(include=['datetime', 'datetimetz']).columns:
    df[col] = df[col].apply(
        lambda x: None if pd.isnull(x) or x == pd.NaT or str(x).strip() in ['NaT', '']
        else x.isoformat() if isinstance(x, pd.Timestamp) else str(x)
    )

# 데이터 확인
df.head()


,DEVICE_ID,SITE_ID,APP_GRP_ID,APP_ID,CUST_ID,LOGIN_YN,UUID,SESS_FLAG,TOKEN,NOTI_FLAG,BMKT_FLAG,APP_VER,OS,OS_VER,DEVICE,DEL_YN,REG_DATE,UPT_DATE,NIGHT_FLAG
0,1,1,5,68,962493,N,48670df1-d2f2-49b7-b213-3afa9b5b8e9d,NNN,bbb16954c640483e95c938320a16af71,Y,Y,6.9.3,A,11.8,OnePlus 9,N,2023-02-01T12:53:31,2024-04-14T14:31:41,Y
1,2,10,2,37,516465,Y,6df2b6ec-fa4f-40d9-9b2d-52179894290d,YYN,f746056915a344bcaf9969fe6348bc44,Y,N,10.1.0,I,10.1,iPhone 13,N,2023-11-09T15:33:30,None,Y
2,3,9,3,88,124538,Y,c09ee61b-6691-4342-afbb-5e9990ce78dd,YYN,81c1d3fd389d4b298212cf14f2fc6c91,N,N,10.9.3,A,10.1,OnePlus 9,Y,2023-03-14T19:55:30,None,None
3,4,10,1,87,128783,N,97a5c8bc-1963-4e9f-9a97-7324bc3f5703,NNN,45d57193d63f49a8a7e7e59bf159e91f,N,N,6.9.4,A,9.0,Google Pixel 6,Y,2024-08-13T10:55:49,None,None
4,5,3,2,22,286542,N,af8275e9-57d9-4c3c-abd3-9333a6abd98e,YYN,None,Y,N,7.8.6,I,13.5,iPhone 13,Y,2023-02-02T04:02:31,None,Y


In [160]:
import snowflake.connector
import pandas as pd

# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='MWAA_API',                # 스노우플레이크 사용자명
    password='ghaTyvld1!',            # 비밀번호
    account='DZ48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_load_WH',      # 스키마 이름
    database='dw_load_db'
)

cur = conn.cursor()

[2025-02-12T16:57:49.828+0900] {connection.py:413} INFO - Snowflake Connector for Python Version: 3.12.2, Python Version: 3.11.10, Platform: Windows-10-10.0.22631-SP0
[2025-02-12T16:57:49.830+0900] {connection.py:1196} INFO - Connecting to GLOBAL Snowflake domain
[2025-02-12T16:57:49.830+0900] {connection.py:1277} INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


In [161]:
# 원본 및 임시 테이블 이름 설정
snowflake_table = "ODS_TMS.TMS_APP_DEVICE_LIST_TEMP"
temp_table = f"{snowflake_table}_TEMP"

# 컬럼 및 PK 설정
columns = [
    "DEVICE_ID", "SITE_ID", "APP_GRP_ID", "APP_ID", "CUST_ID", "LOGIN_YN", "UUID", 
    "SESS_FLAG", "TOKEN", "NOTI_FLAG", "BMKT_FLAG", "APP_VER", "OS", "OS_VER", 
    "DEVICE", "DEL_YN", "REG_DATE", "UPT_DATE", "NIGHT_FLAG"
]
pk_columns = ["DEVICE_ID"]  # PK 컬럼 리스트

# 1️⃣ 임시 테이블 생성
create_temp_table_query = f"""
CREATE TEMPORARY TABLE {temp_table} AS
SELECT * FROM {snowflake_table} WHERE 1=0;
"""  # 빈 임시 테이블 생성
cur.execute(create_temp_table_query)

# 2️⃣ DataFrame 데이터를 Snowflake로 INSERT
insert_query = f"""
INSERT INTO {temp_table} ({", ".join(columns)})
VALUES ({", ".join(["%s"] * len(columns))});
"""

# DataFrame 데이터를 튜플 리스트로 변환 (batch insert)
values = df.where(pd.notnull(df), None).values.tolist()
cur.executemany(insert_query, values)  # batch insert 실행

# 3️⃣ MERGE 실행
merge_condition = " AND ".join([f"target.{col} = source.{col}" for col in pk_columns])

update_set = ", ".join([f"target.{col} = source.{col}" for col in columns if col not in pk_columns])
insert_columns = ", ".join(columns)
insert_values = ", ".join([f"source.{col}" for col in columns])

merge_query = f"""
MERGE INTO {snowflake_table} AS target
USING {temp_table} AS source
ON {merge_condition}
WHEN MATCHED THEN
    UPDATE SET {update_set}
WHEN NOT MATCHED THEN
    INSERT ({insert_columns})
    VALUES ({insert_values});
"""

cur.execute(merge_query)

# 4️⃣ Snowflake 세션 종료
cur.close()
conn.close()

print("Data merged successfully without using S3 stage!")

[2025-02-12T16:57:50.913+0900] {cursor.py:1156} INFO - Number of results in first chunk: 1
[2025-02-12T16:57:51.904+0900] {connection.py:788} INFO - closed
[2025-02-12T16:57:51.939+0900] {connection.py:794} INFO - No async queries seem to be running, deleting session
Data merged successfully without using S3 stage!
